## Lab Assignment Four: Multi-Layer Perceptron

#### Luis Garduno

Dataset : https://www.kaggle.com/muonneutrino/us-census-demographic-data/data


--------------------------------

## 1. Load, Split, & Balance

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.1 Loading Data & Adjustments

In [ ]:
import numpy as np
import pandas as pd

# Load the data into memory & save it to a pandas data frame.
df = pd.read_csv("https://raw.githubusercontent.com/luisegarduno/MachineLearning_Projects/master/Datasets/acs2017_census_tract_data.csv");

# Zip State Name with ID number 
def_state = zip(df['State'].unique(), np.arange(52))

# Remove any observations having missing data.
df = df.dropna(axis=0, how='any')
del df['County']

# Encode State's as integers
df['State'] = pd.factorize(df.State)[0]

df.info()



### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.2 Splitting the Dataset (10 points)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# Divide 'ChildPoverty' into 4 classes
ChildPovertyClass = ['Okay', 'U.S. Poverty Rate', 'Very Poor', 'Extremely Poor']

# this creates a new variable
df['ChildPoverty_Class'] = pd.qcut(df['ChildPoverty'],[0.00,0.17,0.28,0.65,1.0],labels=False)

df['ChildPoverty_Class'].plot(kind='barh',alpha=1.0)
plt.show()

The following resources were used to gain an understanding of where the national poverty was during 2017.
- __[Child Poverty in America 2017: State Analysis](https://www.childrensdefense.org/wp-content/uploads/2018/09/Child-Poverty-in-America-2017-State-Fact-Sheet.pdf)__

- __[Child Poverty in America 2017: National Analysis](https://www.childrensdefense.org/wp-content/uploads/2018/09/Child-Poverty-in-America-2017-National-Fact-Sheet.pdf)__

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume I'm equally interested in the classification performance for each class in the dataset.

# Create X data & y target dataframe's
if 'ChildPoverty' in df:
    y = df['ChildPoverty_Class'].values
    del df['ChildPoverty_Class']
    X = df.to_numpy()

# Notice we only perform fit_transform on X
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Divide the data: 80% Training & 20% Testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

print("Training Set", "\n   - Data Shape:",X_train.shape,"\n   - Target Shape:",y_train.shape)
print("\nTesting Set","\n   - Data Shape:",X_test.shape ,"\n   - Target Shape:",y_test.shape)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.3 Balancing the Data

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA as RandomizedPCA

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', RandomizedPCA(n_components=35,random_state=1)),
                    ('clf', LogisticRegression(class_weight='balanced', random_state=1, max_iter=500))])

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print(classification_report(y_test, y_pred, target_names=ChildPovertyClass))

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %s, Acc: %.3f' % (k+1, score))

print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

- Split 80/20. Balance 80, argue whether 20 should be balanced
- Explain reasoning for selecting this method for balancing the dataset. 
- Should balancing of the dataset be done for both the training & testing set? Explain.

If you do it to the test set, you're misrepresenting the reality of the dataset.
We do it on the training set so that our model doesn't overly classify just one child poverty rate.
But on the testing set we don't want to introduce new data or massage the data on a test set.
For the integrity of the data.

------------------------

## 2. Pre-Processing

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 Quantifying Performance

In [ ]:
import sys
from scipy.special import expit
from sklearn.metrics import accuracy_score

# Using the Two-Layer Perceptron network example (Mini-Batching & Glorant) provided during lecture,
# Let's quantify performance using accuracy. We don't normalize or one-hot encode the data just yet.

# start with a simple base classifier, which can't be fit or predicted it only has internal classes to be used by classes that will subclass it
class MultiLayerPerceptron(object):
    def __init__(self, n_hidden=30,
                 C=0.0, epochs=500, eta=0.001,
                 random_state=None, alpha=0.0,
                 decrease_const=0.0, shuffle=True,
                 minibatches=1, activation='sigmoid',
                 obj_func='quadratic'):
        np.random.seed(random_state)
        self.n_hidden = n_hidden
        self.l2_C = C
        self.epochs = epochs
        self.eta = eta
        self.alpha = alpha
        self.decrease_const = decrease_const
        self.shuffle = shuffle
        self.minibatches = minibatches
        self.activation = activation
        self.obj_func = obj_func

    @staticmethod
    def _encode_labels(y):
        """ Encode labels into one-hot representation """
        onehot = pd.get_dummies(y).values.T
        return onehot
    
    def _initialize_weights(self):
        hidden_layers = self.n_hidden

        if not hasattr(hidden_layers, "__iter__"): hidden_layers = [hidden_layers]

        hidden_layers = list(hidden_layers)
        hidden_layers += [self.n_output_]

        """Initialize weights with small random numbers."""
        weights = []
        w1 = []
        #self.n_features_ = self.n_features_
        n_layers = len(hidden_layers)

        for i, n_hidden in enumerate(hidden_layers):

            if self.activation == 'sigmoid' or self.activation == 'linear':
                w1_num_elems = (self.n_features_ + 1) * n_hidden
                w1 = np.random.uniform(-1.0, 1.0, size=w1_num_elems)
                w1 = w1.reshape(n_hidden, self.n_features_ + 1) # reshape to be W

            elif self.activation == 'relu' or self.activation == 'silu':
                if i == n_layers - 1: init_bound = np.sqrt(0.5 / (n_hidden + self.n_features_ + 1))
                else: init_bound = np.sqrt(6. / (n_hidden + self.n_features_ + 1))

                w1 = np.random.uniform(-init_bound, init_bound, (n_hidden, self.n_features_ + 1))
                w1[:, :1] = 0
                
            self.n_features_ = n_hidden
            weights += [w1]
            
        return weights

    # @staticmethod
    def _activation(self, z):
        if self.activation == 'linear': return z
        if self.activation == 'sigmoid': return expit(z)
        if self.activation == 'relu': return np.maximum(0, z.copy())
        if self.activation == 'silu':
            z = z.copy() * expit(z.copy())
            return z
        
    @staticmethod
    def _add_bias_unit(X, how='column'):
        """ Add bias unit (column or row of 1s) to array at index 0 """
        if how == 'column':
            ones = np.ones((X.shape[0], 1))
            X_new = np.hstack((ones, X))
        elif how == 'row':
            ones = np.ones((1, X.shape[1]))
            X_new = np.vstack((ones, X))
        return X_new
    
    @staticmethod
    def _L2_reg(lambda_, weights):
        """Compute L2-regularization cost"""
        # only compute for non-bias terms
        w = 0
        for w_j in weights:
            w += np.mean(w_j[:, 1:] ** 2)
            
        return (lambda_/2.0) * np.sqrt(w)
        
        #return (lambda_/2.0) * np.sqrt(np.mean(w1[:, 1:] ** 2) + np.mean(w2[:, 1:] ** 2))
    
    def _cost(self, a3, Y_enc, weights):
        """ Get the objective function value """
        if self.obj_func == 'quadratic':
            cost = np.mean((Y_enc - a3) ** 2)
        elif self.obj_func == 'cross_entropy':
            #cost = -np.mean(np.nan_to_num((Y_enc*np.log(a3[-1])+(1-Y_enc)*np.log(1-a3[-1]))))
            #cost = -np.mean(np.nan_to_num((Y_enc*np.log(a3)+(1-Y_enc)*np.log(1-a3))))
            cost = -np.mean(np.nan_to_num((Y_enc*np.log(a3)+(1-Y_enc)*np.log(1-a3))))
        
        L2_term = self._L2_reg(self.l2_C, weights)
        return cost + L2_term
        
    def _feedforward(self, X, weights):
        """ Compute feedforward step """
        a_i = []
        z_i = []
        z = None
        for idx, W in enumerate(weights):
            if idx == 0:
                a1 = self._add_bias_unit(X.T, how='row')
            else:
                a1 = self._activation(z)
                a1 = self._add_bias_unit(a1, how='row')
            z1 = W @ a1
            a_i += [a1]
            z_i += [z1]
            z = z1
        a_out = self._activation(z)
        a_i += [a_out]
        
        return a_i, z_i

    def _get_gradient(self, a, z, Y_enc, weights):
        """ Compute gradient step using backpropagation. """
        # vectorized backpropagation
        gradients = []
        A_output = a[-1]
        
        if self.obj_func == 'quadratic':
            V_last = -2 * (Y_enc - A_output) * A_output * (1-A_output)
        elif self.obj_func == 'cross_entropy':
            V_last = (A_output - Y_enc)
        
        i = 1
        for A, W in zip(a[:-1][::-1], weights[::-1]):
            if i == 1:
                grad = V_last @ A.T         # no bias on final layer
                V1 = (W.T @ V_last)         # back prop the sensitivity
            else:
                grad = V_last[1:,:] @ A.T   # dont back prop sensitivity of bias
                if i != len(weights):
                    V1 = (W.T @ V_last[1:, :])
            
            if len(weights)-1-i >= 0:

                if self.activation == 'linear':
                    V1 = V1

                elif self.activation == 'sigmoid':
                    V1 = A * (1 - A) * V1

                elif self.activation == 'relu':
                    Z1_with_bias = self._add_bias_unit(z[len(weights)-1-i],how='row')
                    V1[Z1_with_bias<=0] = 0

                elif self.activation == 'silu':
                    Z1_with_bias = self._add_bias_unit(z[len(weights)-1-i],how='row')
                    sig = expit(Z1_with_bias.copy())
                    V1 = (A + (1-A) * sig * V1)
                V_last = V1

            # regularize weights that are not bias terms
            grad[:, 1:] += W[:, 1:] * self.l2_C
            gradients.insert(0, grad)
            
            i += 1
        return gradients
    
    def fit(self, X, y, print_progress=False):
        """ Learn weights from training data. With mini-batch"""
        X_data, y_data = X.copy(), y.copy()
        y_enc = self._encode_labels(y)

        # init weights and setup matrices
        self.n_features_ = X_data.shape[1]
        self.n_output_ = y_enc.shape[0]
        self.weights= self._initialize_weights()
        
        W_prev_list = []
        for W in self.weights:
            delta_W_prev = np.zeros(W.shape)
            W_prev_list += [delta_W_prev]

        self.cost_ = []
        self.score_ = []
        # get starting acc
        self.score_.append(accuracy_score(y_data,self.predict(X_data)))
        for i in range(self.epochs):

            # adaptive learning rate
            self.eta /= (1 + self.decrease_const*i)

            if print_progress>0 and (i+1)%print_progress==0:
                sys.stderr.write('\rEpoch: %d/%d' % (i+1, self.epochs))
                sys.stderr.flush()

            if self.shuffle:
                idx_shuffle = np.random.permutation(y_data.shape[0])
                X_data, Y_enc, y_data = X_data[idx_shuffle], y_enc[:, idx_shuffle], y_data[idx_shuffle]

            mini = np.array_split(range(y_data.shape[0]), self.minibatches)
            mini_cost = []
            for idx in mini:
                a, z = self._feedforward(X_data[idx], self.weights)

                cost = self._cost(a[-1], Y_enc[:, idx], self.weights)
                mini_cost.append(cost)

                # compute gradient via backpropagation
                gradients = self._get_gradient(a, z, Y_enc[:,idx], self.weights)

                delta_W_list = []
                for gradient in gradients:
                    delta_W = self.eta * gradient
                    delta_W_list += [delta_W]

            self.cost_.append(mini_cost)
            self.score_.append(accuracy_score(y_data,self.predict(X_data)))

        return self
    
    def get_params(self,deep=True):
        return {"n_hidden" : self.n_hidden,
                "activation" : self.activation,
                "obj_func" : self.obj_func}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def predict(self, X):
        """ Predict class labels """
        A3, _, = self._feedforward(X, self.weights)
        y_pred = np.argmax(A3[-1], axis=0)
        return y_pred

In [ ]:
# Be sure that training converges by graphing the loss function vs. # of epochs.
params = dict(n_hidden=(50,50,50,),
              C=0.1,
              epochs=300,
              eta=0.001,
              random_state=1,
              alpha=0.001,
              decrease_const=0.0001,
              shuffle=True,
              minibatches=50,
              activation='relu',
              obj_func='quadratic')

In [ ]:
%%time 
mlp = MultiLayerPerceptron(**params)
mlp.fit(X_train, y_train, print_progress=10)
yhat = mlp.predict(X_test)
print('Test acc:',accuracy_score(y_test,yhat))

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 Normalizing Data

In [ ]:
# Now normalize the continuous numeric feature data.
# Use the example two-layer perceptron network from the class example & quantify performance using accuracy.

In [ ]:
# Be sure that training converges by graphing the loss function vs. # of epochs.


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 Normalizing & One Hot Encoding the Data

In [ ]:
# Now normalize the continuous numeric feature data AND one hot encode the categorical data.
# Use the example two-layer perceptron network from the class example & quantify performance using accuracy.

In [ ]:
# Be sure that training converges by graphing the loss function vs. # of epochs.

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.4 Comparing the Models

In [ ]:
# Compare the performance of the three models I just trained.

- Are there any meaningful differences in performance?
- In my own words,explain why these models have (or don't have) different performances.
    - _Use one-hot encoding & normalization on the dataset for the remainder of this lab assignment._


------------------

## 3. Modeling

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1 Three-layer Support


In [ ]:
# Add sup for a third layer in the multi-layer perceptron.
# Add sup for saving(& plotting after training is completed) the avg magnitude of the gradient for each layer, for each epoch.

In [ ]:
# Quantify the performance of the model & graph the magnitudes for each layer Vs. the # of epochs.

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.2 Four-layer Support

In [ ]:
# Repeat the previous step, adding support for a fourth layer.

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.3 Five-layer Support

In [ ]:
# Repeat the previous step, adding support for a fifth layer.

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.4 Adaptive Learning Technique w/ five-layer network

In [ ]:
# Implement an adaptive learning technique that was discussed in lecture & use it on the five layer network.
# Don't use AdaM for the adaptive learning technique. 

- Compare the performance of this model w/ & w/o the adaptive learning strategy.


---------------------------

## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4. Adaptive Momentum
- 5000 level student: I have free reign to provide additional analyses.
- One idea: Implement adaptive momentum (AdaM) in the five layer neural network & quantify the performance. 


-----------------------

### Reference

Kaggle. US Census Demographics. https://www.kaggle.com/muonneutrino/us-census-demographic-data/data (Accessed 10-24-2020)